<a href="https://colab.research.google.com/github/ctkerns/cap4630_spring2020/blob/master/hw4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

## Homework 4 - Problem 2
Here is a function I created to find a maximum value in a submatrix.  
The starting max value is -infinity so that matrices with negative values could be supported.

In [0]:
def max_in_mat(mat):
    # Find shape of input mat where m is number of rows and n is number of columns.
    m = len(mat)
    n = len(mat[0])

    # Find the maximum scalar value in the matrix
    max = float('-inf')
    
    for i in range(m):
        for j in range(n):
            if mat[i][j] > max:
                max = mat[i][j]
    
    return max

This is the function from problem 1 which identifies matrices that aren't square then throws an exception.

In [0]:
class InvalidInput(Exception):
    pass

def is_square(mat):
    if len(mat.shape) <= 1:
        if len(mat) != 1:
            raise InvalidInput("One of provided matrices is not a square matrix.")
    elif mat.shape[0] != mat.shape[1]:
        raise InvalidInput("One of provided matrices is not a square matrix.")

Here is a function I made to pad the input matrix with zeros.  
I considered simply shortening the pooling window when it overlapped the edge of the input matrix, but that approach would not support input matrices with negative values.  

To find how many more rows or columns of zeros need to be added be for the width and height of the matrix are divisible by s, the pooling size, I created a formula.  
* * *

If $l$ is the size of the matrix in a certain direction, then
$l\space mod\space s$ is the amount by which the input matrix fills the pooling window in that direction.

$s - l\space mod\space s$ gives the difference between what needs to be filled and what is already filled in that direction. In other words, the number of rows or columns that need to be added.  

There is a caveat to this. If $l$ is divisible by $s$ then the formula simply returns $s$ when it should return 0. To resolve this, I get the remainder of the whole equation so far divided by $s$.  

$(s - l\space mod\space s)\space mod\space s$  

Voila. The number of columns or rows to be added will never exceed the pooling size, so the formula is unchanged except in that edge case I mentioned.



In [0]:
def pad_zeros(mat, s):
    # Find shape of input mat where m is number of rows and n is number of columns.
    m = len(mat)
    n = len(mat[0])

    # Add rows of zeros until the number of rows is divisible by the window height.
    for i in range((s - m%s)%s):
        mat = np.append(mat, np.full((1, n), 0), axis = 0)

    # Add columns of zeros until the number of columns is divisible by the window width.
    for i in range((s - n%s)%s):
        mat = np.insert(mat, n + i, 0, axis = 1)
    
    return mat

And finally, I implemented the function maxpooling2d here.

In [0]:
def maxpooling2d(input_mat, s):
    # Assert that the input matrix is square.
    is_square(input_mat)

    # Pad the input matrix with zeros.
    input_mat = pad_zeros(input_mat, s)

    # Find shape of input mat where m is number of rows and n is number of columns.
    m = len(input_mat)
    n = len(input_mat[0])

    output_mat = []

    for i in range(0, m, s):
        output_mat.append([])
        for j in range(0, n, s):
            max = max_in_mat(input_mat[i:i+s, j:j+s])

            output_mat[int(i/s)].append(max)

    return output_mat